In [1]:
import ROOT as rt
import root_numpy as rtnp
import numpy as np
from matplotlib import pyplot as plt
from histo_utilities import create_TH1D, create_TH2D, std_color_list
from array import array
from prettytable import PrettyTable

import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 1
CMS_lumi.extraText = "     Simulation Preliminary"

donotdelete = []
import csv
import re
import sys

Welcome to JupyROOT 6.13/02


In [2]:
f = open('/home/cms/Desktop/LLP/delayed_jet_analyzer/hlt_2018.py')
content = csv.reader(f)
hlt_paths = []
    
search_term = 'prescales'
line_num = 0
for line in content:
    for ele in line:
        if re.search(search_term, ele):
            if float(line[1]) == 1: #no prescale, highest lumi
                result = re.search('"(.*)"', prev_line[0])
                path = result.group(1)
                if "HLT_" in path:
                    index = path.index('_v')
                    hlt_paths.append(path[:index]) #only save the name without version number
            assert(not(not line[1]==line[2]==line[3]==line[4] and float(line[1])==1)) #check the first four highest lumi have the same prescale
            if line == None:
                print 'no matches found'
    prev_line = line

In [4]:
trigger_names_file = '/home/cms/Desktop/LLP/delayed_jet_analyzer/data/trigger_names_llp_v1.dat'
trigger_names = []
with open(trigger_names_file) as f:
    reader = csv.reader(f, delimiter=" ")
    for line in reader:
        trigger_names.append(line[2])

In [5]:
count = 0
i = len(trigger_names)-1
unprescaled_path_index = []
for path in hlt_paths:
    if path in trigger_names:
        index = trigger_names.index(path)
        unprescaled_path_index.append(index)
    else:
#         print (str(i)+"  " +path)
        i+=1
        count+=1
print(str(count)+" of "+str(len(hlt_paths))+" triggers in 2018 not in trigger_names_llp_v1.dat")

0 of 471 triggers in 2018 not in trigger_names_llp_v1.dat


In [9]:
fpath = {}
tree = {}
#without ISR
fpath['ggh'] = '/home/cms/Desktop/LLP/jetNtuple_RunIISummer16/withoutISR/jetNtuple_glueball/jet_timing_studies_ntuple_ggh_mh125_mx50_pl1000_ev10000.root'
fpath['vbfh'] = '/home/cms/Desktop/LLP/jetNtuple_RunIISummer16/withoutISR/jetNtuple_glueball/jet_timing_studies_ntuple_vbfh_mh125_mx50_pl1000_ev10000.root'
fpath['wh'] = '/home/cms/Desktop/LLP/jetNtuple_RunIISummer16/withoutISR/jetNtuple_glueball/jet_timing_studies_ntuple_wh_mh125_mx50_pl1000_ev10000.root'
fpath['zh'] = '/home/cms/Desktop/LLP/jetNtuple_RunIISummer16/withoutISR/jetNtuple_glueball/jet_timing_studies_ntuple_zh_mh125_mx50_pl1000_ev10000.root'

#withISR
# fpath['ggh'] = '/home/cms/Desktop/LLP/jetNtuple_RunIISummer16/withISR/jet_timing_studies_ntuple_ggh_ISR_mh125_mx50_pl1000_ev100000.root'
# fpath['vbfh'] = '/home/cms/Desktop/LLP/jetNtuple_RunIISummer16/withISR/jet_timing_studies_ntuple_vbfh_ISR_mh125_mx50_pl1000_ev100000.root'



for k,v in fpath.iteritems():
    print k, v
    tree[k] = rtnp.root2array(v)
    #tree[k] = rtnp.tree2array(v)
    #jet_max_pt_array = [np.max(x) for x in tree[k]["jetPt"]]
    print k
    

zh /home/cms/Desktop/LLP/jetNtuple_RunIISummer16/withoutISR/jetNtuple_glueball/jet_timing_studies_ntuple_zh_mh125_mx50_pl1000_ev10000.root
zh
ggh /home/cms/Desktop/LLP/jetNtuple_RunIISummer16/withoutISR/jetNtuple_glueball/jet_timing_studies_ntuple_ggh_mh125_mx50_pl1000_ev10000.root
ggh
vbfh /home/cms/Desktop/LLP/jetNtuple_RunIISummer16/withoutISR/jetNtuple_glueball/jet_timing_studies_ntuple_vbfh_mh125_mx50_pl1000_ev10000.root
vbfh
wh /home/cms/Desktop/LLP/jetNtuple_RunIISummer16/withoutISR/jetNtuple_glueball/jet_timing_studies_ntuple_wh_mh125_mx50_pl1000_ev10000.root
wh


In [7]:
# label['ggHm50ct100mmWISR'] = 'ggH M_{X} = 50 GeV, c#tau = 100 mm'
legend = {
    'zee':'Z(ee)',
    'zmumu':'Z(#mu#mu)',
    'znunu':'Z(#nu#nu)',
    'zhad':'Z(hadronic)',
    'wenu':'W(e#nu)',
    'wmunu':'W(#mu#nu)',
    'whad':'W(hadronic)',
    'vbfh':'vbfH',
    'ggh':'ggH',
}
lumi = (37.80+44.98+63.67) * 1000 #in pb, CMS recorded 2016 + 2017 + 2018
br = {
    'zee':0.033632,
    'zmumu':0.033662,
    'znunu':0.20000,
    'zhad':0.69911,
    'wenu':0.1071,
    'wmunu':0.1063,
    'whad':0.6741,
    'vbfh':1,
    'ggh':1,
}


cs = {
    'zee':0.88,
    'zmumu':0.88,
    'znunu':0.88,
    'zhad':0.88,
    'wenu':1.37,
    'wmunu':1.37,
    'whad':1.37,
    'vbfh':3.78,
    'ggh':44.1,
} #in pb


n_mc = {}
for k,v in cs.items():
    n_mc[k] = lumi*br[k]*cs[k]
    print(n_mc[k])


21488.16915
135249.06465
25775.2
4334.357632
90098.50036
21327.65995
4338.223912
6458445.0
553581.0


In [23]:
l1_prescaled = [239,240,241,242,243] #PFJets
l1_prescaled = l1_prescaled + [229,230,231,232,233] #AK8PFJet
l1_prescaled = l1_prescaled + [249,250,251,252,253] #HLT_DiPFJetAve200
l1_prescaled = l1_prescaled + [351,352,353,354] #HLT_Ele8_CaloIdL_TrackIdL_IsoVL_PFJet30
l1_prescaled = l1_prescaled + [511,512,513,514]
l1_prescaled = l1_prescaled + [258,259,260,261]

l1_prescaled = l1_prescaled +[149,150] #not in wbm
golden_path = {
    'zee':362,
    'zmumu':136,#double muon triggers are different for 2018 and 2016, the one used in 2018 doesn't exist in 2016
    'znunu':310,
    'zhad':310,
    'wenu':87, #84 is in hlt.py but not in wbm
    'wmunu':196,
    'whad':310,
    'vbfh':310,
    'ggh':310,   
}

In [24]:
# inclusive trigger efficiency * acceptance

trigger_decision = {}
table = PrettyTable()
table.field_names = ["Process","Trigger path","Produced","Efficiency (%)","# of Events"]
prescale_threshold = {
     'zee':0.5,
    'zmumu':0.56,
    'znunu':0.05,
    'zhad':0.077,
    'wenu':0.43,
    'wmunu':0.52,
    'whad':0.077,
    'vbfh':0.15,
    'ggh':0.03,   
}
prescale_threshold = {
     'zee':0.98,
    'zmumu':0.98,
    'znunu':0.15,
    'zhad':0.7,
    'wenu':0.9,
    'wmunu':0.9,
    'whad':0.7,
    'vbfh':0.7,
    'ggh':0.05,   
}
for k,v in cs.items():
    trigger_decision[k] = {}
    for tr in unprescaled_path_index:
        trigger_decision[k][tr] = []
for k,T in tree.items():
    gParticleId = T['gParticleId']
    gParticlePt = T['gParticlePt']
    gParticleMotherId = T['gParticleMotherId']
    gParticleMotherIndex = T['gParticleMotherIndex']
    gParticleStatus = T['gParticleStatus']
    hlt = T['HLTDecision']
    if k == 'zh':
        keys = ['zee','zmumu','znunu','zhad']
        
        for i in range(len(T['isQCD'])): #loop over all events
            mumu = np.logical_and(np.logical_and(np.absolute(gParticleId[i]) == 13, np.absolute(gParticleMotherId[i]) == 23), gParticleStatus[i] == 1 )
            ee = np.logical_and(np.logical_and(np.absolute(gParticleId[i]) == 11, np.absolute(gParticleMotherId[i]) == 23), gParticleStatus[i] == 1 )
            nunu_temp = np.logical_or(np.logical_or(np.absolute(gParticleId[i]) == 12,np.absolute(gParticleId[i]) == 14),np.absolute(gParticleId[i]) == 16)
            nunu = np.logical_and(np.logical_and(nunu_temp, np.absolute(gParticleMotherId[i]) == 23), gParticleStatus[i] == 1 )
#             had = np.logical_not(np.logical_or(nunu,np.logical_or(mumu,ee)))
            had = np.logical_and(np.logical_and(np.absolute(gParticleId[i]) <= 6, np.absolute(gParticleMotherId[i]) == 23), gParticleStatus[i] == 23 )
            mother_id = gParticleMotherId[i][np.absolute(gParticleId[i]) == 13] 
            for tr in unprescaled_path_index:
                if np.count_nonzero(mumu) == 2:
                    trigger_decision['zmumu'][tr].append(hlt[i][tr])
                    assert(np.count_nonzero(ee)==np.count_nonzero(nunu)==0)
                elif np.count_nonzero(ee) == 2:
                    trigger_decision['zee'][tr].append(hlt[i][tr])
                    assert(np.count_nonzero(mumu)==np.count_nonzero(nunu)==0)
                elif np.count_nonzero(nunu) == 2:
                    trigger_decision['znunu'][tr].append(hlt[i][tr])
                    assert(np.count_nonzero(ee)==np.count_nonzero(mumu)==0)
                elif np.count_nonzero(had) == 2:
                    trigger_decision['zhad'][tr].append(hlt[i][tr])
        # checking the br makes sense    
#         print("zmumu br:"+str(len(trigger_decision['zmumu'][177])*1.0/len(T['isQCD'])))
#         print("zee br:"+str(len(trigger_decision['zee'][177])*1.0/len(T['isQCD'])))
#         print("znunu br:"+str(len(trigger_decision['znunu'][177])*1.0/len(T['isQCD'])))
#         print("zhad br:"+str(len(trigger_decision['zhad'][177])*1.0/len(T['isQCD'])))
        for key in keys:
            trigger_or = np.zeros(np.array(trigger_decision[key][unprescaled_path_index[0]]).shape)
            max_eff = 0
            max_eff_tr = 0
            golden = trigger_decision[key][golden_path[key]]
            for tr in unprescaled_path_index:
                if tr not in l1_prescaled:
                    eff = np.mean(trigger_decision[key][tr])
                    if eff < prescale_threshold[key] and eff > 0.0:
                        # check if overlapping is the same
                        check_or = np.logical_or(trigger_decision[key][tr] , golden)
                        if not np.array_equal(golden, check_or):
                            print(key,trigger_names[tr],eff)

                        trigger_or += np.array(trigger_decision[key][tr])
                        if eff > max_eff:
                            max_eff = eff
                            max_eff_tr = tr

            eff = np.count_nonzero(trigger_or)*1.0/len(trigger_or)
            num = eff* n_mc[key]
            if eff > 0.05:
                table.add_row([key,trigger_names[max_eff_tr],n_mc[key], round(100*max_eff,2),round(max_eff*n_mc[key],2)])
                table.add_row([key,"Or",n_mc[key], round(100*eff,2),round(num,2)])
    elif k == 'wh':
        keys = ['wenu','wmunu','whad']

        for i in range(len(T['isQCD'])): #loop over all events
            mu = np.logical_and(np.logical_and(np.absolute(gParticleId[i]) == 13, np.absolute(gParticleMotherId[i]) == 24), gParticleMotherIndex[i] == 1 )
            e = np.logical_and(np.logical_and(np.absolute(gParticleId[i]) == 11, np.absolute(gParticleMotherId[i]) == 24), gParticleMotherIndex[i] == 1 )
            nu_temp = np.logical_or(np.logical_or(np.absolute(gParticleId[i]) == 12,np.absolute(gParticleId[i]) == 14),np.absolute(gParticleId[i]) == 16)
            nu = np.logical_and(np.logical_and(nu_temp, np.absolute(gParticleMotherId[i]) == 24), gParticleMotherIndex[i] == 1 )
            had = np.logical_and(np.logical_and(np.absolute(gParticleId[i]) <= 6, np.absolute(gParticleMotherId[i]) == 24), gParticleMotherIndex[i] == 1 )
            for tr in unprescaled_path_index:
                if np.count_nonzero(mu) == 1 and np.count_nonzero(nu) == 1:
                    trigger_decision['wmunu'][tr].append(hlt[i][tr])
                    assert(np.count_nonzero(e)==np.count_nonzero(had)==0)
                elif np.count_nonzero(e) == 1 and np.count_nonzero(nu) == 1:
                    trigger_decision['wenu'][tr].append(hlt[i][tr])
                    assert(np.count_nonzero(mu)==np.count_nonzero(had)==0)
                elif np.count_nonzero(had) == 2:
                    trigger_decision['whad'][tr].append(hlt[i][tr])
                    assert(np.count_nonzero(mu)==np.count_nonzero(e)==np.count_nonzero(nu)==0)

#         # checking the br makes sense    
#         print("wmunu br:"+str(len(trigger_decision['wmunu'][177])*1.0/len(T['isQCD'])))
#         print("wenu br:"+str(len(trigger_decision['wenu'][177])*1.0/len(T['isQCD'])))
#         print("whad br:"+str(len(trigger_decision['whad'][177])*1.0/len(T['isQCD'])))
        for key in keys:
            trigger_or = np.zeros(np.array(trigger_decision[key][unprescaled_path_index[0]]).shape)
            max_eff = 0
            max_eff_tr = 0
            golden = trigger_decision[key][golden_path[key]]
            for tr in unprescaled_path_index:
                if tr not in l1_prescaled:
                    eff = np.mean(trigger_decision[key][tr])
                    if eff < prescale_threshold[key] and eff > 0.0:
                        # check if overlapping is the same
                        check_or = np.logical_or(trigger_decision[key][tr] , golden)
                        if not np.array_equal(golden, check_or):
                            print(key,trigger_names[tr],eff)

                        trigger_or += np.array(trigger_decision[key][tr])
                        if eff > max_eff:
                            max_eff = eff
                            max_eff_tr = tr
            eff = np.count_nonzero(trigger_or)*1.0/len(trigger_or)
            num = eff* n_mc[key]
            if eff > 0.05:
                table.add_row([key,trigger_names[max_eff_tr],n_mc[key], round(100*max_eff,2),round(max_eff*n_mc[key],2)])
                table.add_row([key,"Or",n_mc[key], round(100*eff,2),round(num,2)])
            
    elif k == 'vbfh' or k == 'ggh':
        key = k
        for i in range(len(T['isQCD'])): #loop over all events
            for tr in unprescaled_path_index:
                trigger_decision[k][tr].append(hlt[i][tr])
        trigger_or = np.zeros(np.array(trigger_decision[key][unprescaled_path_index[0]]).shape)
        max_eff = 0
        max_eff_tr = 0
            
        golden = trigger_decision[key][golden_path[key]]
        for tr in unprescaled_path_index:
            if tr not in l1_prescaled:
                eff = np.mean(trigger_decision[key][tr])
                if eff < prescale_threshold[key] and eff > 0.0:
                    # check if overlapping is the same
                    check_or = np.logical_or(trigger_decision[key][tr] , golden)
                    if not np.array_equal(golden, check_or):
                        print(key,trigger_names[tr],eff)

                    trigger_or += np.array(trigger_decision[key][tr])
                    if eff > max_eff:
                        max_eff = eff
                        max_eff_tr = tr

        
        
        eff = np.count_nonzero(trigger_or)*1.0/len(trigger_or)
        num = eff* n_mc[key]        
        if eff > 0.05:
            table.add_row([key,trigger_names[max_eff_tr],n_mc[key], round(100*max_eff,2),round(max_eff*n_mc[key],2)])
            table.add_row([key,"Or",n_mc[key], round(100*eff,2),round(num,2)])
                    

                
print(table)

('zee', 'HLT_DoubleEle33_CaloIdL_MW', 0.28113879003558717)
('zee', 'HLT_Mu3_PFJet40', 0.021352313167259787)
('zee', 'HLT_DoublePhoton85', 0.02491103202846975)
('zee', 'HLT_Ele30_WPTight_Gsf', 0.7153024911032029)
('zee', 'HLT_Ele32_WPTight_Gsf', 0.708185053380783)
('zee', 'HLT_DiPFJetAve60_HFJEC', 0.017793594306049824)
('zee', 'HLT_PFHT250', 0.1387900355871886)
('zee', 'HLT_PFMET120_PFMHT120_IDTight', 0.12099644128113879)
('zee', 'HLT_PFMETNoMu120_PFMHTNoMu120_IDTight', 0.12099644128113879)
('zee', 'HLT_MonoCentralPFJet80_PFMETNoMu120_PFMHTNoMu120_IDTight', 0.09608540925266904)
('zee', 'HLT_Ele23_Ele12_CaloIdL_TrackIdL_IsoVL', 0.5338078291814946)
('zee', 'HLT_Photon22', 0.9395017793594306)
('zee', 'HLT_Ele50_CaloIdVT_GsfTrkIdT_PFJet165', 0.08540925266903915)
('zee', 'HLT_Ele115_CaloIdVT_GsfTrkIdT', 0.11743772241992882)
('zee', 'HLT_Ele250_CaloIdVT_GsfTrkIdT', 0.0035587188612099642)
('zee', 'HLT_IsoTrackHB', 0.10676156583629894)
('zee', 'HLT_IsoTrackHE', 0.046263345195729534)
('zmumu', '

# Overall Efficiency Plot